In [20]:
from pyspark.sql.functions import *
df = spark.read.table('DimCustomer')
df2 = df.select("CustomerKey", "NumberCarsOwned", "FirstName", "LastName", "DateFirstPurchase") \
        .where(df.YearlyIncome == 130000) \
        .withColumn("FullName", concat(df.FirstName, lit(" "), df.LastName)) \
        .withColumn("FormattedDate", date_format(df.DateFirstPurchase, "EEEE d MMMM yyyy")) \
        .sort(df.NumberCarsOwned.desc(), df.CustomerKey.asc())
display(df2)

StatementMeta(, a30eb9a3-ea05-40c7-9434-59f9b11e98bf, 22, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c5496704-bd77-4573-903e-277ef60ea752)

In [28]:
df3 = df.groupBy("EnglishOccupation") \
        .avg("YearlyIncome") \
        .withColumnRenamed("avg(YearlyIncome)", "AverageIncome") \
        .withColumn("AverageIncome", col("AverageIncome").cast("decimal(10,2)")) \
        .where("AverageIncome > 70000") \
        .orderBy(desc("AverageIncome"))
display(df3)

StatementMeta(, a30eb9a3-ea05-40c7-9434-59f9b11e98bf, 30, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, bdf2b464-8b60-4a09-9e71-2829bfd89654)

In [34]:
df3 = df.groupBy("EnglishOccupation") \
        .agg(avg("YearlyIncome").alias("AverageIncome")) \
        .where("AverageIncome > 70000") \
        .orderBy(desc("AverageIncome"))
df3 = df3.select(df3.AverageIncome.cast("int")) # 这里记得int要用引号
display(df3)

StatementMeta(, a30eb9a3-ea05-40c7-9434-59f9b11e98bf, 36, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 854ad210-8f8d-421a-8c32-97b63ee30599)

In [41]:
%%sql
SELECT CustomerKey, NumberCarsOwned, FirstName, LastName, DateFirstPurchase,
       concat(FirstName, " ", LastName) AS FullName,
       date_format(DateFirstPurchase, "EEEE d MMMM yyyy") AS ForamattedDate
FROM DimCustomer
WHERE YearlyIncome = 130000
ORDER BY NumberCarsOwned DESC, CustomerKey



StatementMeta(, a30eb9a3-ea05-40c7-9434-59f9b11e98bf, 43, Finished, Available, Finished)

<Spark SQL result set with 512 rows and 7 fields>

In [47]:
%%sql
SELECT EnglishOccupation, 
       sum(TotalChildren) as Children,
       cast(avg(YearlyIncome) as int) as AverageIncome
FROM DimCustomer
GROUP BY EnglishOccupation
HAVING AverageIncome > 70000
-- 部分数据库（如 PostgreSQL、Spark SQL 新版本）不支持 HAVING 用别名，就需要使用 cast(avg(YearlyIncome) as int) > 70000
ORDER BY Children

StatementMeta(, a30eb9a3-ea05-40c7-9434-59f9b11e98bf, 49, Finished, Available, Finished)

<Spark SQL result set with 2 rows and 3 fields>